In [1]:
import os
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
from tensorflow.keras import layers, models, Input, Model
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras import backend as K

In [2]:
mask_path = './ISIC_Masks/ISIC2018_Task1_Training_GroundTruth_x2'
img_path = 'ISIC_Images/ISIC2018_Task1-2_Training_Input_x2'

In [3]:
def img_process_norm(img):
    img_raw = tf.io.read_file(img)
    img = tf.image.decode_jpeg(img_raw, channels=3)
    img = tf.image.resize(img, [256, 256])
    img /= 255.0
    return img

def mask_process_norm(mask):
    mask_raw = tf.io.read_file(mask)
    mask = tf.image.decode_png(mask_raw, channels=1)
    mask = tf.image.resize(mask, [256, 256])
    mask /= 255.0
    return mask

def implot_show(ds):
    for img, mask in ds:
        print(img.shape)
        display_list = [img, mask]
        plt.figure(figsize=(18, 18))
        for i in range(2):
            print(display_list[i].shape)
            plt.subplot(1, 2, i+1)
            plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
            plt.axis('off')
        plt.show()

In [4]:
def load_data():

    masks = os.listdir(mask_path)
    masks = [os.path.join(mask_path, path) for path in masks[1:-1]]
    masks = tf.data.Dataset.from_tensor_slices(masks)
    mask_input_ds = masks.map(mask_process_norm, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    
    images = os.listdir(img_path)
    images = [os.path.join(img_path, path) for path in images[1:-1]]
    images = tf.data.Dataset.from_tensor_slices(images)
    image_input_ds = images.map(img_process_norm, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    
    image_ds = tf.data.Dataset.zip((image_input_ds, mask_input_ds))
    
    implot_show(image_ds.take(2))
    
    return image_ds

In [5]:
def split_train_test_val(image_ds):
    size = 2594
    
    train_size = int(0.8 * size)
    val_size = int(0.1 * size)
    test_size = int(0.1 * size)
    
    train_image = image_ds.take(train_size)
    test_image = image_ds.skip(train_size)
    val_image = test_image.take(val_size)
    test_image = test_image.skip(val_size)
    
    return train_image, val_image, test_image

In [6]:
image_ds = load_data()

train_image, val_image, test_image = split_train_test_val(image_ds)

FileNotFoundError: [WinError 3] The system cannot find the path specified: './ISIC_Masks/ISIC2018_Task1_Training_GroundTruth_x2'

In [7]:
def UNet_context_module(filters, inp, layer_name):
    # Each context_module consists of two 3x3 conv layers and a dropout(0.3) in between.
    
    x1 = layers.Conv2D(filters, kernel_size =3, padding = 'same')(inp)
    x1 = layers.BatchNormalization()(x1)
    x1 = layers.LeakyReLU(alpha=0.01)(x1)
    x1 = layers.Dropout(.3)(x1)
    x2 = layers.Conv2D(filters, kernel_size =3, padding = 'same')(x1)
    x2 = layers.BatchNormalization()(x2)
    x2 = layers.LeakyReLU(alpha=0.01)(x2)

    return x2
    
def UNet_upsampling_module(filters, inp):
    # ...It is like a layer that combines the UpSampling2D and Conv2D layers into one layer. 
    
    # what twice means in paper (Answer from Piazza: kernel size 2 by 2)?
    x1 = layers.UpSampling2D(size=(2,2))(inp)
    x2 = layers.Conv2D(filters, kernel_size =3, padding = 'same')(x1)
    x2 = layers.BatchNormalization()(x2)
    x2 = layers.LeakyReLU(alpha=0.01)(x2)
    
    return x2
    
    
def UNet_localization_module(filters, inp):
    # A localization module consists of a 3x3x3 convolution followed by a 1x1x1 convolution that halves the
    # number of feature maps.
    
    x1 = layers.Conv2D(filters, kernel_size =3, padding = 'same')(inp)
    x1 = layers.BatchNormalization()(x1)
    x1 = layers.LeakyReLU(alpha=0.01)(x1)
    
    x1 = layers.Dropout(.3)(x1)
    x2 = layers.Conv2D(filters, kernel_size =1, padding = 'same')(x1)
    x2 = layers.BatchNormalization()(x2)
    x2 = layers.LeakyReLU(alpha=0.01)(x2)
    
    return x2

In [8]:
def Improved_UNet_model():
    
    filters = 16
    input_layer = Input((256,256,3))
    
    # block 1:
    block1_x1 = layers.Conv2D(filters, kernel_size =3, padding = 'same')(input_layer)
    #block1_x1 = layers.BatchNormalization()(block1_x1)
    block1_x1 = layers.LeakyReLU(alpha=0.01)(block1_x1)
    #block1_x1 = layers.Dropout(0.3)(block1_x1)
    
    block1_x2 = UNet_context_module(filters, block1_x1, "context_module1")
    
    output_b1 = layers.Add()([block1_x1, block1_x2])
    
    
    # block 2:
    block2_x1 = layers.Conv2D(filters*2, kernel_size =3, strides = 2, padding = 'same')(output_b1)
    #block2_x1 = layers.BatchNormalization()(block2_x1)
    block2_x1 = layers.LeakyReLU(alpha=0.01)(block2_x1)
    #block2_x1 = layers.Dropout(0.3)(block2_x1)
    
    block2_x2 = UNet_context_module(filters*2, block2_x1, "context_module2")
    
    output_b2 = layers.Add()([block2_x1, block2_x2])
    
    
    # block 3:
    block3_x1 = layers.Conv2D(filters*4, kernel_size =3, strides = 2, padding = 'same')(output_b2)
    #block3_x1 = layers.BatchNormalization()(block3_x1)
    block3_x1 = layers.LeakyReLU(alpha=0.01)(block3_x1)
    #block3_x1 = layers.Dropout(0.3)(block3_x1)
    
    block3_x2 = UNet_context_module(filters*4, block3_x1, "context_module3")
    
    output_b3 = layers.Add()([block3_x1, block3_x2])
    
    
    # block 4:
    block4_x1 = layers.Conv2D(filters*8, kernel_size =3, strides = 2, padding = 'same')(output_b3)
    #block4_x1 = layers.BatchNormalization()(block4_x1)
    block4_x1 = layers.LeakyReLU(alpha=0.01)(block4_x1)
    #block4_x1 = layers.Dropout(0.3)(block4_x1)
    
    block4_x2 = UNet_context_module(filters*8, block4_x1, "context_module4")
    
    output_b4 = layers.Add()([block4_x1, block4_x2])
    
    
    # block 5:
    block5_x1 = layers.Conv2D(filters*16, kernel_size =3, strides = 2, padding = 'same')(output_b4)
    #block5_x1 = layers.BatchNormalization()(block5_x1)
    block5_x1 = layers.LeakyReLU(alpha=0.01)(block5_x1)
    #block5_x1 = layers.Dropout(0.3)(block5_x1)
    
    block5_x2 = UNet_context_module(filters*16, block5_x1, "context_module5")
    
    output_b5 = layers.Add()([block5_x1, block5_x2])
    
    
    # up_block 6:
    block6_x1 = UNet_upsampling_module(filters*8, output_b5)
    # connection
    output_b6 = layers.concatenate([output_b4, block6_x1])
    
    
    # up_block 7:
    block7_x1 = UNet_localization_module(filters*8, output_b6)
    block7_x2 = UNet_upsampling_module(filters*4, block7_x1)
    # connection
    output_b7 = layers.concatenate([output_b3, block7_x2])
    
    
    # up_block 8:
    block8_x1 = UNet_localization_module(filters*4, output_b7)
    block8_x2 = UNet_upsampling_module(filters*2, block8_x1)
    # connection
    output_b8 = layers.concatenate([output_b2, block8_x2])
    
    
    # up_block 9:
    block9_x1 = UNet_localization_module(filters*2, output_b8)
    block9_x2 = UNet_upsampling_module(filters, block9_x1)
    # connection
    output_b9 = layers.concatenate([output_b1, block9_x2])
    
    # upscale
    segmentation_1 = layers.Conv2D(1, kernel_size =3, padding = 'same')(block7_x1)
    segmentation_1 = layers.UpSampling2D(size=(8,8))(segmentation_1)
    segmentation_2 = layers.Conv2D(1, kernel_size =3, padding = 'same')(block8_x1)
    segmentation_2 = layers.UpSampling2D(size=(4,4))(segmentation_2)
    final_block_output = layers.Conv2D(1, kernel_size =3, padding = 'same')(output_b9)
    
    output = layers.Add()([segmentation_1, segmentation_2, final_block_output])
    #output = layers.BatchNormalization()(output)
    output = layers.Activation('sigmoid')(output)
    
    improved_unet_model = Model(input_layer, output, name="improved_unet_model")
    improved_unet_model.summary()
    
    return improved_unet_model

In [9]:
improved_unet_model = Improved_UNet_model()

Model: "improved_unet_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 16) 448         input_1[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 256, 256, 16) 0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 16) 2320        leaky_re_lu[0][0]                
________________________________________________________________________________

In [10]:
def dice_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
    return K.mean( (2. * intersection + smooth) / (union + smooth), axis=0)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

In [11]:
train_size = len(list(train_image))
val_size = len(list(val_image))
test_size = len(list(test_image))

BATCH_SIZE=32
STEPS_PER_EPOCH =train_size//BATCH_SIZE
train_image = train_image.batch(BATCH_SIZE).repeat()
val_image = val_image.batch(BATCH_SIZE)
test_image = test_image.batch(1)

NameError: name 'train_image' is not defined

In [ ]:
from tensorflow.keras.optimizers import SGD
opt = SGD(lr=0.2)

initial_learning_rate = 0.005
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=1000,
    decay_rate=0.985,
    staircase=True)
opt = SGD(learning_rate=lr_schedule)


improved_unet_model.compile(optimizer=opt, loss=dice_coef_loss, metrics=[dice_coef])

VALIDATION_STEPS = val_size//BATCH_SIZE

model_history = improved_unet_model.fit(train_image,steps_per_epoch=STEPS_PER_EPOCH ,epochs=200, validation_data=val_image)

Epoch 1/200
 1/64 [..............................] - ETA: 5:36:21 - loss: 0.7091 - dice_coef: 0.2909